In [2]:
import pandas as pd
import random
#from google.colab import drive
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/annemoll/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

# Set the device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
MAX_TITLE_LEN = 20

articles_path = 'Data/articles.parquet'
behaviors_train_path = 'Data/behaviors_train.parquet'
behaviors_val_path = 'Data/behaviors_val.parquet'
history_train_path = 'Data/history_train.parquet'
history_val_path = 'Data/history_val.parquet'

# Load data from Parquet files
articles = pd.read_parquet(articles_path)
train_behaviors = pd.read_parquet(behaviors_train_path)
val_behaviors = pd.read_parquet(behaviors_val_path)
history_train = pd.read_parquet(history_train_path)
history_val = pd.read_parquet(history_val_path)

# Create a news dict and  article_id to a unique index
news = {}
newsindex = {'NULL': 0}  # Add a NULL key for padding
for idx, row in articles.iterrows():
    article_id = row['article_id']
    title = row['title'].lower()
    tokenized_title = word_tokenize(title)

    news[article_id] = tokenized_title
    newsindex[article_id] = len(newsindex)

# Summary
print("Number of unique articles:", len(news))
print("Example article ID and tokenized title:")
for k, v in list(news.items())[:3]:  # Print first 3 articles
    print(f"Article ID: {k}, Tokenized Title: {v}")

print("\nTotal articles indexed in newsindex:", len(newsindex), "\n")


# Helper function to sample negative examples
def newsample(array, ratio):
    if len(array) == 0:
        return []
    if ratio > len(array):
        return random.sample(array * (ratio // len(array) + 1), ratio)
    else:
        return random.sample(array, ratio)

# Sampling configuration
npratio = 4  # Number of negative samples per positive sample
MAX_HISTORY_LEN = 50

# Function to process behaviors data
def process_behaviors(behaviors, newsindex, history_data=None):
    train_candidate = []
    train_label = []
    train_user_his = []

    # Build a user history dictionary from history data if provided
    user_history = {}
    if history_data is not None:
        for _, row in history_data.iterrows():
            user_history[row['user_id']] = [newsindex.get(aid, 0) for aid in row['article_id_fixed']]

    for _, row in behaviors.iterrows():
        user_id = row['user_id']

        # Clicked articles (positive examples)
        clicked = [newsindex.get(aid, 0) for aid in row['article_ids_clicked'] if aid in newsindex]
        # Non-clicked articles (negative examples)
        inview = set(row['article_ids_inview'])
        non_clicked = [newsindex.get(aid, 0) for aid in inview if aid in newsindex and aid not in row['article_ids_clicked']]

        # User history
        if user_id in user_history:
            clickids = user_history[user_id][-MAX_HISTORY_LEN:]
        else:
            clickids = clicked[-MAX_HISTORY_LEN:]

        for pos_doc in clicked:
            neg_docs = newsample(non_clicked, npratio)
            candidates = neg_docs + [pos_doc]
            labels = [0] * npratio + [1]

            # Shuffle candidates and labels
            shuffle_indices = list(range(len(candidates)))
            random.shuffle(shuffle_indices)
            shuffled_candidates = [candidates[i] for i in shuffle_indices]
            shuffled_labels = [labels[i] for i in shuffle_indices]

            # Append training data
            train_candidate.append(shuffled_candidates)
            train_label.append(shuffled_labels)
            train_user_his.append(clickids + [0] * (MAX_HISTORY_LEN - len(clickids)))

    return train_candidate, train_label, train_user_his

# Process train behaviors
train_candidate, train_label, train_user_his = process_behaviors(train_behaviors, newsindex, history_train)
val_candidate, val_label, val_user_his = process_behaviors(val_behaviors, newsindex, history_val)

""" # Process validation behaviors
val_candidate, val_user_his, val_labels, val_index = [], [], [], []
for _, row in val_behaviors.iterrows():
    user_id = row['user_id']
    clicked = [newsindex.get(aid, 0) for aid in row['article_ids_clicked'] if aid in newsindex]
    #inview = set(row['article_ids_inview'])
    #non_clicked = [newsindex.get(aid, 0) for aid in inview if aid in newsindex and aid not in row['article_ids_clicked']]

    user_history = clicked[-MAX_HISTORY_LEN:]
    user_history = user_history + [0] * (MAX_HISTORY_LEN - len(user_history))


    start_idx = len(val_candidate)

    for aid in row['article_ids_inview']:
        if aid in newsindex:
            val_candidate.append(newsindex[aid])
            val_user_his.append(user_history)
            val_labels.append(1 if aid in row['article_ids_clicked'] else 0)

    end_idx = len(val_candidate)
    val_index.append([start_idx, end_idx]) """

# Print summary
print("Training Data:")
print("Number of training candidates:", len(train_candidate))
print("Number of training labels:", len(train_label))
print("Number of user histories:", len(train_user_his))

print("\nValidation Data:")
print("Number of validation candidates:", len(val_candidate))
print("Number of validation labels:", len(val_label))
print("Number of user histories:", len(val_user_his))
#print("Number of users in validation data:", len(val_index))
#print("Validation index ranges (start and end for each user):")
#for idx, (start, end) in enumerate(val_index):
  #  print(f"  User {idx + 1}: Start = {start}, End = {end}, Number of candidates = {end - start}")


Number of unique articles: 11777
Example article ID and tokenized title:
Article ID: 3037230, Tokenized Title: ['ishockey-spiller', ':', 'jeg', 'troede', 'jeg', 'skulle', 'dø']
Article ID: 3044020, Tokenized Title: ['prins', 'harry', 'tvunget', 'til', 'dna-test']
Article ID: 3057622, Tokenized Title: ['rådden', 'kørsel', 'på', 'blå', 'plader']

Total articles indexed in newsindex: 11778 

Training Data:
Number of training candidates: 24888
Number of training labels: 24888
Number of user histories: 24888

Validation Data:
Number of validation candidates: 25505
Number of validation labels: 25505
Number of user histories: 25505


In [6]:
# Save Train Data
train_df = pd.DataFrame({
    'candidate': train_candidate,
    'label': train_label,
    'user_his': train_user_his
})
train_df.to_parquet('Data/train_data.parquet', index=False)

# Save Validation Data
val_df = pd.DataFrame({
    'candidate': val_candidate,
    'label': val_label,
    'user_his': val_user_his
})
val_df.to_parquet('Data/val_data.parquet', index=False)

# Print Confirmation
print("Train and Validation data saved as Parquet files:")
print("  - train_data.parquet")
print("  - val_data.parquet")


Train and Validation data saved as Parquet files:
  - train_data.parquet
  - val_data.parquet


In [7]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

# Build vocabulary from tokenized titles
vocab = {'<PAD>': 0}  # Start with a padding token
for tokens in news.values():
    for word in tokens:
        if word not in vocab:
            vocab[word] = len(vocab)

# Convert titles to token indices
news_tensor = {}
for article_id, tokens in news.items():
    token_indices = [vocab[word] for word in tokens]  # Convert words to token indices

    # Truncate or pad to MAX_TITLE_LEN
    if len(token_indices) > MAX_TITLE_LEN:
        token_indices = token_indices[:MAX_TITLE_LEN]  # Truncate if too long
    else:
        token_indices += [0] * (MAX_TITLE_LEN - len(token_indices))  # Pad with zeros

    news_tensor[newsindex[article_id]] = torch.tensor(token_indices, dtype=torch.long)
    news_tensor[0] = torch.zeros(MAX_TITLE_LEN, dtype=torch.long)

news_tensors_list = [tensor[:MAX_TITLE_LEN] for tensor in news_tensor.values()]  # Truncate to MAX_TITLE_LEN
news_tensors_padded = pad_sequence(news_tensors_list, batch_first=True, padding_value=vocab['<PAD>'])

# Example Output
print("Vocabulary size:", len(vocab))
print("Shape of padded news tensors:", news_tensors_padded.shape)

Vocabulary size: 16003
Shape of padded news tensors: torch.Size([11778, 20])


In [8]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, data, news_tensor, max_history_len=50):
        """
        Args:
            data (pd.DataFrame): DataFrame with 'candidate', 'label', and 'user_his'
            news_tensor (dict): Dictionary mapping article_id to tokenized title tensors
            max_history_len (int): Maximum length of user history (default: 50)
        """
        self.data = data
        self.news_tensor = news_tensor
        self.max_history_len = max_history_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Candidate titles: List of tokenized article titles (positive + negatives)
        candidate_ids = row['candidate']
        candidate_titles = torch.stack([self.news_tensor[aid] for aid in candidate_ids])

        # Labels: Positive (1) and negative (0) labels for candidates
        labels = torch.tensor(row['label'], dtype=torch.float)

        # User history: List of clicked articles converted to tokenized titles
        user_his_ids = row['user_his']
        user_his_titles = torch.stack([self.news_tensor[aid] for aid in user_his_ids])

        # Pad user history if it's shorter than max length
        if len(user_his_titles) < self.max_history_len:
            padding = torch.zeros((self.max_history_len - len(user_his_titles), candidate_titles.shape[1]), dtype=torch.long)
            user_his_titles = torch.cat((user_his_titles, padding), dim=0)

        return candidate_titles, user_his_titles, labels


In [9]:
from torch.utils.data import DataLoader

# Load train and validation data
train_data = pd.read_parquet('Data/train_data.parquet')
val_data = pd.read_parquet('Data/val_data.parquet')

# Initialize the Dataset for train and validation
train_dataset = NewsDataset(train_data, news_tensor, max_history_len=50)
val_dataset = NewsDataset(val_data, news_tensor, max_history_len=50)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Fetch a batch to test
for candidate_titles, user_his_titles, labels in train_loader:
    print("Candidate Titles Shape:", candidate_titles.shape)  # (batch_size, num_candidates, MAX_TITLE_LEN)
    print("User History Shape:", user_his_titles.shape)      # (batch_size, max_history_len, MAX_TITLE_LEN)
    print("Labels Shape:", labels.shape)                    # (batch_size, num_candidates)
    break

Candidate Titles Shape: torch.Size([32, 5, 20])
User History Shape: torch.Size([32, 50, 20])
Labels Shape: torch.Size([32, 5])


In [10]:
class NewsEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim=300, num_heads=20, attention_hidden_dim=200):
        """
        Args:
            vocab_size (int): Size of the vocabulary
            embedding_dim (int): Dimension of word embeddings
            max_title_len (int): Maximum length of article titles
        """
        super(NewsEncoder, self).__init__()

        # Word Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # Single-Head Self-Attention
        self.multihead_attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)

        # Additive Attention Network
        self.additive_attention_query = nn.Parameter(torch.randn(attention_hidden_dim))  # Query vector
        self.additive_attention_fc1 = nn.Linear(embedding_dim, attention_hidden_dim)
        self.additive_attention_fc2 = nn.Linear(attention_hidden_dim, 1)

        # Linear Layer to output fixed-size vector
        self.fc = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, title_tokens):
        """
        Args:
            title_tokens (Tensor): Shape (batch_size, max_title_len)
                                   - Tokenized and padded title tensors
        Returns:
            Tensor: Fixed-size vector representing the article (batch_size, embedding_dim)
        """
        # Step 1: Word Embedding
        embedded = self.embedding(title_tokens)  # Shape: (batch_size, max_title_len, embedding_dim)

        # Step 2: Multi-Head Self-Attention
        attn_output, _ = self.multihead_attention(embedded, embedded, embedded)  # Shape: (batch_size, max_title_len, embedding_dim)

        # Step 3: Additive Attention
        additive_weights = torch.tanh(self.additive_attention_fc1(attn_output))  # Shape: (batch_size, max_title_len, attention_hidden_dim)
        additive_scores = self.additive_attention_fc2(additive_weights).squeeze(-1)  # Shape: (batch_size, max_title_len)

        # Compute attention weights (softmax over words)
        attention_weights = torch.softmax(additive_scores, dim=1)  # Shape: (batch_size, max_title_len)

        # Weighted sum of the attention outputs
        weighted_sum = torch.sum(attn_output * attention_weights.unsqueeze(-1), dim=1)  # Shape: (batch_size, embedding_dim)

        # Step 4: Linear Transformation
        output_vector = self.fc(weighted_sum)  # Shape: (batch_size, embedding_dim)

        return output_vector


In [11]:
# Define parameters
VOCAB_SIZE = len(vocab)  # Vocabulary size
EMBEDDING_DIM = 300      # Dimension of word embeddings
MAX_TITLE_LEN = 20       # Length of padded titles

# Initialize the News Encoder
news_encoder = NewsEncoder(vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM)

# Fetch a batch of candidate titles
for candidate_titles, user_his_titles, labels in train_loader:
    # Input shape: (batch_size, num_candidates, MAX_TITLE_LEN)
    batch_size, num_candidates, title_len = candidate_titles.shape
    _, max_history_len, _ = user_his_titles.shape

    # Reshape to merge batch_size and num_candidates
    candidate_titles_reshaped = candidate_titles.view(-1, title_len)  # Shape: (batch_size * num_candidates, MAX_TITLE_LEN)

    # Pass through the News Encoder
    candidate_vectors = news_encoder(candidate_titles_reshaped)  # Shape: (batch_size * num_candidates, EMBEDDING_DIM)

    # Reshape back to original batch_size and num_candidates
    candidate_vectors = candidate_vectors.view(batch_size, num_candidates, EMBEDDING_DIM)

    # Reshape user history titles to merge batch_size and max_history_len
    user_his_titles_reshaped = user_his_titles.view(-1, title_len)  # Shape: (batch_size * max_history_len, MAX_TITLE_LEN)

    # Pass through News Encoder
    user_his_vectors = news_encoder(user_his_titles_reshaped)  # Shape: (batch_size * max_history_len, EMBEDDING_DIM)

    # Reshape back to original batch_size and max_history_len
    user_his_vectors = user_his_vectors.view(batch_size, max_history_len, EMBEDDING_DIM)  # Shape: (batch_size, max_history_len, EMBEDDING_DIM)

    print("Shape of candidate_vectors:", candidate_vectors.shape)  # Expected: (batch_size, num_candidates, EMBEDDING_DIM)
    print("Shape of user_his_vectors:", user_his_vectors.shape)    # Expected: (batch_size, max_history_len, EMBEDDING_DIM)
    break

Shape of candidate_vectors: torch.Size([32, 5, 300])
Shape of user_his_vectors: torch.Size([32, 50, 300])


In [12]:
class UserEncoder(nn.Module):
    def __init__(self, embedding_dim=300, num_heads=20, attention_hidden_dim=200):
        """
        Args:
            embedding_dim (int): Dimension of the article embeddings
            num_heads (int): Number of attention heads in multi-head attention
            attention_hidden_dim (int): Dimension of the query vector in additive attention
        """
        super(UserEncoder, self).__init__()

        # Multi-Head Self-Attention
        self.multihead_attention = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                         num_heads=num_heads,
                                                         batch_first=True)

        # Additive Attention
        self.additive_attention_query = nn.Parameter(torch.randn(attention_hidden_dim))  # Learnable query vector
        self.additive_attention_fc1 = nn.Linear(embedding_dim, attention_hidden_dim)
        self.additive_attention_fc2 = nn.Linear(attention_hidden_dim, 1)

    def forward(self, user_his_vectors):
        """
        Args:
            user_his_vectors (Tensor): Shape (batch_size, max_history_len, embedding_dim)
                                        - Representations of clicked articles
        Returns:
            Tensor: User representation vector (batch_size, embedding_dim)
        """
        # Step 1: Multi-Head Self-Attention
        attn_output, _ = self.multihead_attention(user_his_vectors, user_his_vectors, user_his_vectors)
        # Shape: (batch_size, max_history_len, embedding_dim)

        # Step 2: Additive Attention
        additive_weights = torch.tanh(self.additive_attention_fc1(attn_output))  # Shape: (batch_size, max_history_len, attention_hidden_dim)
        additive_scores = self.additive_attention_fc2(additive_weights).squeeze(-1)  # Shape: (batch_size, max_history_len)

        # Compute attention weights (softmax over user history)
        attention_weights = torch.softmax(additive_scores, dim=1)  # Shape: (batch_size, max_history_len)

        # Weighted sum of the attention outputs
        user_vector = torch.sum(attn_output * attention_weights.unsqueeze(-1), dim=1)  # Shape: (batch_size, embedding_dim)

        return user_vector

In [13]:
# Define parameters
EMBEDDING_DIM = 300      # Dimension of article embeddings
NUM_HEADS = 20           # Number of attention heads
ATTENTION_HIDDEN_DIM = 200  # Query vector dimension for additive attention

# Initialize User Encoder
user_encoder = UserEncoder(embedding_dim=EMBEDDING_DIM, num_heads=NUM_HEADS, attention_hidden_dim=ATTENTION_HIDDEN_DIM)

# Fetch a batch of user history vectors
for candidate_titles, user_his_titles, labels in train_loader:
    # Reshape user history titles to merge batch_size and max_history_len
    user_his_titles_reshaped = user_his_titles.view(-1, MAX_TITLE_LEN)  # Shape: (batch_size * max_history_len, MAX_TITLE_LEN)

    # Pass through News Encoder to get user history vectors
    user_his_vectors = news_encoder(user_his_titles_reshaped)  # Shape: (batch_size * max_history_len, EMBEDDING_DIM)

    # Reshape back to (batch_size, max_history_len, EMBEDDING_DIM)
    batch_size, max_history_len, _ = user_his_titles.shape
    user_his_vectors = user_his_vectors.view(batch_size, max_history_len, EMBEDDING_DIM)

    # Pass user history vectors through User Encoder
    user_vectors = user_encoder(user_his_vectors)  # Shape: (batch_size, EMBEDDING_DIM)

    print("Shape of user_his_vectors:", user_his_vectors.shape)  # (batch_size, max_history_len, EMBEDDING_DIM)
    print("Shape of user_vectors:", user_vectors.shape)         # (batch_size, EMBEDDING_DIM)
    break

Shape of user_his_vectors: torch.Size([32, 50, 300])
Shape of user_vectors: torch.Size([32, 300])


In [14]:
class NRMSModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=300, num_heads=20, attention_hidden_dim=200, max_history_len=50):
        """
        Args:
            vocab_size (int): Size of the vocabulary
            embedding_dim (int): Dimension of word embeddings
            num_heads (int): Number of attention heads in multi-head attention
            attention_hidden_dim (int): Dimension of the query vector in additive attention
            max_history_len (int): Maximum length of user history
            max_title_len (int): Maximum length of article titles
        """
        super(NRMSModel, self).__init__()

        # News Encoder for both candidate and user history articles
        self.news_encoder = NewsEncoder(vocab_size=vocab_size,
                                         embedding_dim=embedding_dim,
                                         num_heads=num_heads,
                                         attention_hidden_dim=attention_hidden_dim)

        # User Encoder to encode user history into a single user representation vector
        self.user_encoder = UserEncoder(embedding_dim=embedding_dim,
                                         num_heads=num_heads,
                                         attention_hidden_dim=attention_hidden_dim)

    def forward(self, candidate_titles, user_his_titles):
        """
        Args:
            candidate_titles (Tensor): Shape (batch_size, num_candidates, max_title_len)
                                       - Tokenized and padded titles of candidate articles
            user_his_titles (Tensor): Shape (batch_size, max_history_len, max_title_len)
                                       - Tokenized and padded titles of user clicked articles
        Returns:
            Tensor: Click scores for each candidate article (batch_size, num_candidates)
        """
        batch_size, num_candidates, max_title_len = candidate_titles.shape
        _, max_history_len, _ = user_his_titles.shape

        # -------------------
        # Process Candidate Articles
        # -------------------
        # Reshape candidates to merge batch_size and num_candidates
        candidate_titles_reshaped = candidate_titles.view(-1, max_title_len)  # Shape: (batch_size * num_candidates, max_title_len)

        # Encode candidate articles
        candidate_vectors = self.news_encoder(candidate_titles_reshaped)  # Shape: (batch_size * num_candidates, embedding_dim)

        # Reshape back to original batch_size and num_candidates
        candidate_vectors = candidate_vectors.view(batch_size, num_candidates, -1)  # Shape: (batch_size, num_candidates, embedding_dim)

        # -------------------
        # Process User History
        # -------------------
        # Reshape user history titles to merge batch_size and max_history_len
        user_his_titles_reshaped = user_his_titles.view(-1, max_title_len)  # Shape: (batch_size * max_history_len, max_title_len)

        # Encode user history articles
        user_his_vectors = self.news_encoder(user_his_titles_reshaped)  # Shape: (batch_size * max_history_len, embedding_dim)

        # Reshape back to original batch_size and max_history_len
        user_his_vectors = user_his_vectors.view(batch_size, max_history_len, -1)  # Shape: (batch_size, max_history_len, embedding_dim)

        # Encode user history into a single user representation vector
        user_vectors = self.user_encoder(user_his_vectors)  # Shape: (batch_size, embedding_dim)

        # -------------------
        # Compute Click Scores
        # -------------------
        # Dot product between user vector and candidate vectors
        click_scores = torch.bmm(candidate_vectors, user_vectors.unsqueeze(-1)).squeeze(-1)  # Shape: (batch_size, num_candidates)

        return click_scores


In [15]:
# Initialize the NRMS Model
nrms_model = NRMSModel(vocab_size=VOCAB_SIZE,
                       embedding_dim=EMBEDDING_DIM,
                       num_heads=NUM_HEADS,
                       attention_hidden_dim=ATTENTION_HIDDEN_DIM,
                       max_history_len=50)

# Fetch a batch from the DataLoader
for candidate_titles, user_his_titles, labels in train_loader:
    # Pass through NRMS model
    click_scores = nrms_model(candidate_titles, user_his_titles)  # Shape: (batch_size, num_candidates)

    print("Shape of click_scores:", click_scores.shape)  # Expected: (batch_size, num_candidates)
    print("Example click scores:", click_scores[0])     # Print scores for the first user in the batch
    break


Shape of click_scores: torch.Size([32, 5])
Example click scores: tensor([-0.0106, -0.0193, -0.0229, -0.0140, -0.0102],
       grad_fn=<SelectBackward0>)


In [16]:
# Define a function to decode tokenized titles back into text
def decode_title(token_indices, vocab):
    reverse_vocab = {idx: word for word, idx in vocab.items()}  # Reverse the vocabulary
    return " ".join([reverse_vocab[idx] for idx in token_indices if idx != 0])  # Ignore padding tokens (0)

# Fetch a batch from the DataLoader
for candidate_titles, user_his_titles, labels in train_loader:
    # Pass through NRMS model
    click_scores = nrms_model(candidate_titles, user_his_titles)  # Shape: (batch_size, num_candidates)

    # Loop through the batch
    batch_size, num_candidates, max_title_len = candidate_titles.shape
    for i in range(batch_size):
        print(f"\nUser {i + 1}:")
        print("-" * 30)

        # Decode and print each candidate title, its label, and its score
        for j in range(num_candidates):
            title_tokens = candidate_titles[i, j].cpu().numpy()  # Get token indices for the title
            title_text = decode_title(title_tokens, vocab)  # Decode the title back into text
            label = labels[i, j].item()  # Get the label
            score = click_scores[i, j].item()  # Get the click score

            print(f"Candidate {j + 1}:")
            print(f"  Title: {title_text}")
            print(f"  Label: {label}")
            print(f"  Click Score: {score:.4f}")
    break



User 1:
------------------------------
Candidate 1:
  Title: græder sig selv i søvn
  Label: 1.0
  Click Score: -0.0104
Candidate 2:
  Title: smed vigtige point : kan skrive historie
  Label: 0.0
  Click Score: -0.0144
Candidate 3:
  Title: forsvaret varsler støjgener og militærkøretøjer på vejene i nordsjælland
  Label: 0.0
  Click Score: -0.0199
Candidate 4:
  Title: utroligt : disse svampe 'taler ' sammen efter regnskyl
  Label: 0.0
  Click Score: -0.0142
Candidate 5:
  Title: sejr sender dortmund på mesterskabskurs
  Label: 0.0
  Click Score: -0.0138

User 2:
------------------------------
Candidate 1:
  Title: luksus-konkurs undersøges : offentlige millioner væltede ind
  Label: 1.0
  Click Score: -0.0024
Candidate 2:
  Title: dmi : teltfesterne er reddet
  Label: 0.0
  Click Score: -0.0060
Candidate 3:
  Title: vanvidspris : det dyreste danske nogensinde
  Label: 0.0
  Click Score: -0.0228
Candidate 4:
  Title: knivstikkeri i fælledparken : en anholdt og sigtet
  Label: 0.0
  Cl

In [17]:
import torch.optim as optim

# Define parameters
NUM_EPOCHS = 3          # Number of epochs
LEARNING_RATE = 1e-4    # Learning rate
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the model, loss, and optimizer
nrms_model = NRMSModel(vocab_size=VOCAB_SIZE,
                       embedding_dim=EMBEDDING_DIM,
                       num_heads=NUM_HEADS,
                       attention_hidden_dim=ATTENTION_HIDDEN_DIM,
                       max_history_len=50).to(DEVICE)

criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.Adam(nrms_model.parameters(), lr=LEARNING_RATE)  # Optimizer

# Training Loop
for epoch in range(NUM_EPOCHS):
    nrms_model.train()  # Set model to training mode
    total_loss = 0

    for batch_idx, (candidate_titles, user_his_titles, labels) in enumerate(train_loader):
        # Move data to the appropriate device (GPU/CPU)
        candidate_titles = candidate_titles.to(DEVICE)  # Shape: (batch_size, num_candidates, max_title_len)
        user_his_titles = user_his_titles.to(DEVICE)    # Shape: (batch_size, max_history_len, max_title_len)
        labels = torch.argmax(labels, dim=1).to(DEVICE)  # Convert labels to indices, Shape: (batch_size)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        click_scores = nrms_model(candidate_titles, user_his_titles)  # Shape: (batch_size, num_candidates)

        # Compute loss
        loss = criterion(click_scores, labels)  # CrossEntropyLoss expects (scores, target_indices)
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print progress for every 100 batches
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Print epoch loss
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}] - Average Loss: {avg_loss:.4f}")

print("Training complete!")


Epoch [1/3], Batch [1/778], Loss: 1.6085
Epoch [1/3], Batch [101/778], Loss: 1.5199
Epoch [1/3], Batch [201/778], Loss: 1.5525
Epoch [1/3], Batch [301/778], Loss: 1.4732
Epoch [1/3], Batch [401/778], Loss: 1.5538
Epoch [1/3], Batch [501/778], Loss: 1.5504
Epoch [1/3], Batch [601/778], Loss: 1.6320
Epoch [1/3], Batch [701/778], Loss: 1.5083
Epoch [1/3] - Average Loss: 1.5365
Epoch [2/3], Batch [1/778], Loss: 1.4675
Epoch [2/3], Batch [101/778], Loss: 1.6041
Epoch [2/3], Batch [201/778], Loss: 1.5301
Epoch [2/3], Batch [301/778], Loss: 1.4145
Epoch [2/3], Batch [401/778], Loss: 1.4568
Epoch [2/3], Batch [501/778], Loss: 1.3658
Epoch [2/3], Batch [601/778], Loss: 1.4816
Epoch [2/3], Batch [701/778], Loss: 1.4121
Epoch [2/3] - Average Loss: 1.4664
Epoch [3/3], Batch [1/778], Loss: 1.4108
Epoch [3/3], Batch [101/778], Loss: 1.4033
Epoch [3/3], Batch [201/778], Loss: 1.5279
Epoch [3/3], Batch [301/778], Loss: 1.4058
Epoch [3/3], Batch [401/778], Loss: 1.3647
Epoch [3/3], Batch [501/778], Los

In [23]:
from sklearn.metrics import accuracy_score, roc_auc_score

def validate_model(model, val_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    all_scores = []

    with torch.no_grad():  # No gradient computation during evaluation
        for batch_idx, (candidate_titles, user_his_titles, labels) in enumerate(val_loader):
            # Move data to the appropriate device
            candidate_titles = candidate_titles.to(device)  # Shape: (batch_size, num_candidates, max_title_len)
            user_his_titles = user_his_titles.to(device)    # Shape: (batch_size, max_history_len, max_title_len)
            labels = torch.argmax(labels, dim=1).to(device)  # Convert labels to indices, Shape: (batch_size)

            # Forward pass
            click_scores = model(candidate_titles, user_his_titles)  # Shape: (batch_size, num_candidates)

            # Predicted indices (max score per candidate set)
            preds = torch.argmax(click_scores, dim=1).cpu().numpy()  # Shape: (batch_size)

            # Append true labels and predictions for metrics
            all_labels.extend(labels.cpu().numpy())  # True labels
            all_preds.extend(preds)  # Predicted labels
            all_scores.extend(torch.softmax(click_scores, dim=1).cpu().numpy())  # Softmax probabilities

    print(f"Shape of all_labels: {len(all_labels)}")  # Should match the total number of validation samples
    print(f"Shape of all_scores: {len(all_scores), len(all_scores[0])}")  # Should be (n_samples, n_classes)

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_scores, multi_class="ovr")  # Use full softmax scores

    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation AUC: {auc:.4f}")
    return accuracy, auc

In [24]:
# Test the trained model
validate_model(nrms_model, val_loader, DEVICE)


Shape of all_labels: 25505
Shape of all_scores: (25505, 5)
Validation Accuracy: 0.2205
Validation AUC: 0.5598


(0.22050578317976868, 0.5598350127392537)

In [26]:
torch.save(nrms_model.state_dict(), "/content/drive/MyDrive/DTU/Kandidat/Semester 9/Deep learning/Final project/Data/nrms_model.pth")

In [27]:
nrms_model = NRMSModel(vocab_size=VOCAB_SIZE,
                       embedding_dim=EMBEDDING_DIM,
                       num_heads=NUM_HEADS,
                       attention_hidden_dim=ATTENTION_HIDDEN_DIM,
                       max_history_len=50).to(DEVICE)
nrms_model.load_state_dict(torch.load("/content/drive/MyDrive/DTU/Kandidat/Semester 9/Deep learning/Final project/Data/nrms_model.pth"))
nrms_model.eval()  # Set model to evaluation mode


<ipython-input-27-3ba86ca2f9e7>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nrms_model.load_state_dict(torch.load("/content/drive/MyDrive/DTU/Kandidat/Semester 9/Deep l

NRMSModel(
  (news_encoder): NewsEncoder(
    (embedding): Embedding(16003, 300, padding_idx=0)
    (multihead_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
    )
    (additive_attention_fc1): Linear(in_features=300, out_features=200, bias=True)
    (additive_attention_fc2): Linear(in_features=200, out_features=1, bias=True)
    (fc): Linear(in_features=300, out_features=300, bias=True)
  )
  (user_encoder): UserEncoder(
    (multihead_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
    )
    (additive_attention_fc1): Linear(in_features=300, out_features=200, bias=True)
    (additive_attention_fc2): Linear(in_features=200, out_features=1, bias=True)
  )
)

In [25]:
# Inspect click probabilities
for batch_idx, (candidate_titles, user_his_titles, labels) in enumerate(val_loader):
    candidate_titles = candidate_titles.to(DEVICE)
    user_his_titles = user_his_titles.to(DEVICE)
    labels = torch.argmax(labels, dim=1).to(DEVICE)

    # Forward pass
    click_scores = nrms_model(candidate_titles, user_his_titles)
    softmax_probs = torch.softmax(click_scores, dim=1)

    for i in range(candidate_titles.size(0)):  # Iterate over the batch
        print(f"\nUser {i + 1}:")
        print(f"True Label: {labels[i].item()}")
        print(f"Predicted Click Probabilities: {softmax_probs[i].cpu().detach().numpy()}")
    break  # Remove this to inspect the full dataset



User 1:
True Label: 0
Predicted Click Probabilities: [0.3590598  0.03104732 0.28306085 0.06200203 0.26483008]

User 2:
True Label: 0
Predicted Click Probabilities: [0.1983009  0.19637354 0.04171867 0.5255394  0.03806753]

User 3:
True Label: 4
Predicted Click Probabilities: [0.2089105  0.2769646  0.15376191 0.20774192 0.15262102]

User 4:
True Label: 1
Predicted Click Probabilities: [0.18161696 0.09691916 0.3208212  0.188273   0.21236964]

User 5:
True Label: 0
Predicted Click Probabilities: [0.18593705 0.27698404 0.36308143 0.08882122 0.08517627]

User 6:
True Label: 2
Predicted Click Probabilities: [0.20987754 0.07774509 0.2166143  0.33537588 0.16038722]

User 7:
True Label: 2
Predicted Click Probabilities: [0.16927657 0.0018787  0.4383322  0.10109434 0.2894182 ]

User 8:
True Label: 4
Predicted Click Probabilities: [0.15353468 0.27252963 0.14342271 0.13682663 0.2936864 ]

User 9:
True Label: 4
Predicted Click Probabilities: [0.23732632 0.16111258 0.14569937 0.17779927 0.2780625 ]

